In [ ]:
import os
import requests
from flask import Flask, request, jsonify
import nest_asyncio
 
# Permitir múltiples bucles de eventos en Jupyter
nest_asyncio.apply()
 
# Crear la aplicación Flask
app = Flask(__name__)
 
# URL del modelo servido por MLflow, configurable a través de variables de entorno
MODEL_URL = os.getenv("RegresionLineal", "http://127.0.0.1:5001/invocations")
 
@app.route("/", methods=["GET"])
def home():
    """Endpoint de prueba para verificar que la API está en funcionamiento."""
    return jsonify({"message": "Flask está funcionando y listo para recibir predicciones."})
 
@app.route("/predict", methods=["POST"])
def predict():
    """Endpoint para realizar predicciones enviando datos al modelo servido en MLflow."""
    try:
        # Leer datos del cuerpo de la solicitud
        data = request.get_json()
        if not data or "instances" not in data:
            return jsonify({"error": "Solicitud inválida, falta 'instances'."}), 400
 
        # Enviar datos al modelo desplegado en MLflow
        response = requests.post(MODEL_URL, json=data)
        response.raise_for_status()  # Verificar si hubo algún error en la solicitud
        predictions = response.json()
        return jsonify({"predictions": predictions})
    except requests.exceptions.RequestException as e:
        return jsonify({"error": f"Error al conectarse con el modelo en {MODEL_URL}: {str(e)}"}), 500
 
# Ejecutar el servidor Flask en Jupyter Notebook
if __name__ == "__main__":
    from werkzeug.serving import run_simple
run_simple("127.0.0.1", 8000, app)

 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
127.0.0.1 - - [14/Jan/2025 22:48:30] "GET / HTTP/1.1" 200 -
